#CSE 101: Computer Science Principles
####Stony Brook University
####Kevin McDonnell (ktm@cs.stonybrook.edu)
##Module 19: Data Representation and Compression



### Note

* See the course website for the [lecture slides](https://drive.google.com/file/d/1lvKesW38RFRZsok_pWrou6zrhfvC54hC/view?usp=sharing), which are separate from this Colab file




### Huffman Coding: the `PriorityQueue` and `Node` Classes

See the [lecture slides](https://drive.google.com/file/d/1lvKesW38RFRZsok_pWrou6zrhfvC54hC/view?usp=sharing) for information about Huffman coding and the class definitions given below. Note that you are not responsible for understanding the code below.

In [ ]:
class PriorityQueue:
    def __init__(self):
        self._q = []

    def __repr__(self):
        return "[" + ", ".join(map(str, self._q)) + "]"

    def __len__(self):
        return len(self._q)

    def insert(self, x):
        i = 0
        while i < len(self._q):
            if x < self._q[i]: break
            i += 1
        self._q.insert(i, x)
        return self

    def pop(self):
        if len(self._q) > 0:
            return self._q.pop(0)
        else:
            return None

    def clear(self):
        self._q = []

    def __getitem__(self, x):
        return self._q[x]

    def __iter__(self):
        for x in self._q:
            yield x

class Node:
    def __init__(self, arg1, arg2):
        self.id = 'node%d' % Node._id
        if type(arg1) == Node and type(arg2) == Node:
            self._left = arg1
            self._right = arg2
            self._char = None
            self._freq = arg1._freq + arg2._freq
        else:
            self._char = arg1
            self._freq = arg2
            self._left = self._right = None
        Node._id += 1

    _id = 0

    def __repr__(self):
        if self.is_leaf():
            return "( %s: %.3f )" % (self._char, self._freq)
        else:
            return "( %.3f %s %s )" % (self._freq, str(self._left), str(self._right))

    def __lt__(self, rhs):
        return self._freq < rhs._freq

    def is_leaf(self):
        return self._char is not None

    def char(self):
        return self._char

    def freq(self):
        return self._freq

    def left_child(self):
        return self._left

    def right_child(self):
        return self._right

### Huffman Coding: Main Algorithm

Download the files [`hafreq.txt`](https://drive.google.com/file/d/1990CuBInwfWBv2nLjNtl7uN8RoSBvGPL/view?usp=sharing) and [`hvfreq.txt`](https://drive.google.com/file/d/1Vu910-41t4UszTRhMczzqODWMU_rCr56/view?usp=sharing), and then upload then to Colab before attempting to run the code below.

In [ ]:
import heapq

def init_queue(a):
    p = PriorityQueue()
    for x in a:
        p.insert(Node(x, a[x]))
    return p

def build_tree(filename):
    pq = init_queue(read_frequencies(filename))
    while len(pq) > 1:
        n1 = pq.pop()  # remove 1st element
        n2 = pq.pop()  # remove 2nd element
        pq.insert(Node(n1, n2))
    return pq[0]

def assign_codes(node, code=None, prefix=''):
    if code is None:
        code = dict()
    if node.is_leaf():
        code[node._char] = prefix
    else:
        assign_codes(node._left, code, prefix + '0')
        assign_codes(node._right, code, prefix + '1')
    return code

def read_frequencies(filename):
    a = {}
    with open(filename) as freqfile:
        for line in freqfile:
            if line[0] == '\n' or line[0] == '#':
                continue
            letter, freq = line.split()
            a[letter] = float(freq)
    return a

def huffman_encode(word, encodings):
    result = ''
    for letter in word:
        result += encodings[letter]
    return result

def huffman_decode(encoded, decodings):
    result = ''
    while len(encoded) > 0:
        for i in range(1, len(encoded) + 1):
            if encoded[:i] in decodings.keys():
                result += decodings[encoded[:i]]
                encoded = encoded[i:]
                break
    return result

### Huffman Coding: Example #1

See the [lecture slides](https://drive.google.com/file/d/1lvKesW38RFRZsok_pWrou6zrhfvC54hC/view?usp=sharing) for details about this example.

In [ ]:
import pprint

vt = build_tree('hvfreq.txt')
huffman_codes = assign_codes(vt)
print('Code dictionary:')
pprint.pprint(huffman_codes)

### Huffman Coding: Example #2

See the [lecture slides](https://drive.google.com/file/d/1lvKesW38RFRZsok_pWrou6zrhfvC54hC/view?usp=sharing) for details about this example.

In [ ]:
at = build_tree('hafreq.txt')
huffman_codes = assign_codes(at)
print('Code dictionary:')
pprint.pprint(huffman_codes)

reversed_codes = {huffman_codes[key]: key for key in huffman_codes.keys()}

print(f'"MAUI" encoded: {huffman_encode("MAUI", huffman_codes)}')
print(f'Decoded messaged: {huffman_decode("110001001101111", reversed_codes)}')

In [ ]:
vt = build_tree('hvfreq.txt')
huffman_codes = assign_codes(vt)
print('Code dictionary:')
pprint.pprint(huffman_codes)